# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Setup and Imports

In [1]:
# ! pip install -q "datasets==2.15.0"

In [2]:
pip install -U transformers accelerate peft bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import numpy as np
import torch
import os
import shutil

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    # BitsAndBytesConfig,
    TrainingArguments,
    Trainer, 
    DataCollatorWithPadding
)

from peft import LoraConfig, get_peft_model

from sklearn.metrics import accuracy_score, f1_score

root_dir = "./tmp"
# root_dir = "./../../../../data/GenAI/02_genai_fundamentals/project2/results"

training_artifacts_dir = 'training_artifacts'
archive_zip = 'archive'

foundation_model_name = "distilbert-base-uncased"

foundation_model_path = os.path.join(root_dir, training_artifacts_dir, "foundation_model", "model")
foundation_model_output_path = os.path.join(root_dir, training_artifacts_dir, "foundation_model", "output")


lora_model_path = os.path.join(root_dir, "lora_model", "model")
lora_model_output_path = os.path.join(root_dir, "lora_model", "output")

qlora_model_path = os.path.join(root_dir, "qlora_model", "model")
qlora_model_output_path = os.path.join(root_dir, "qlora_model", "output")
                                
                                
batch_size = 16
train_epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Clearn up

In [5]:
# Check if the folder exists before attempting to delete
folder_to_delete = root_dir
if os.path.exists(folder_to_delete):
    try:
        shutil.rmtree(folder_to_delete)
        print(f"Folder '{folder_to_delete}' and its contents deleted successfully.")
    except OSError as e:
        print(f"Error: {folder_to_delete} : {e.strerror}")
else:
    print(f"Folder '{folder_to_delete}' does not exist.")

Folder './tmp' and its contents deleted successfully.


## Load Dataset

In [6]:
# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [7]:
# dataset = load_dataset("sms_spam", split="train")
# dataset = dataset.select(range(3))

# dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=23)


# splits = ["train", "test"]

# # View the dataset characteristics
# dataset["train"]

In [8]:
# Inspect the first example
dataset["train"][0]

{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n',
 'label': 1}

## Pre-process datasets

In [9]:
tokenizer = AutoTokenizer.from_pretrained(foundation_model_name)

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )


# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [10]:
# Inspect the available columns in the dataset
tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [11]:
foundation_model = AutoModelForSequenceClassification.from_pretrained(
    foundation_model_name,
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in foundation_model.parameters():
    param.requires_grad = True

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(foundation_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


In [14]:
def TrainAndValidate(model, model_output_path, model_path):
    model.to(device)
    
    # Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir=model_output_path,
            # Set the learning rate
            learning_rate=2e-5,
            # Set the per device train batch size and eval batch size
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            # Evaluate and save the model after each epoch
            # evaluation_strategy="epoch",
            eval_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=train_epochs,
            weight_decay=0.01,
            load_best_model_at_end=True,
        ),
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    
    
    model.to(device)
    trainer.evaluate()
    
    print("saving model", model_path)
    model.save_pretrained(model_path)

## Train Foundation Model

In [15]:
TrainAndValidate(foundation_model, foundation_model_output_path, foundation_model_path)

/tmp/ipykernel_504/601428653.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.042949,0.988341
2,0.051800,0.051471,0.987444


saving model ./tmp/training_artifacts/foundation_model/model


## Evaluate the model

In [16]:
# foundation_model.to(device)
# trainer.evaluate()

In [17]:
# foundation_model.save_pretrained(foundation_model_path)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [18]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "k_lin", "v_lin"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

lora_model = get_peft_model(foundation_model, lora_config)

# lora_model.print_trainable_parameters()

lora_trainer = TrainAndValidate(lora_model, lora_model_output_path, lora_model_path)


/tmp/ipykernel_504/601428653.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.044815,0.988341
2,0.021200,0.044745,0.988341


saving model ./tmp/lora_model/model


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [19]:
# # Saving the model
# lora_model.save_pretrained(lora_model_path)

## Performing Bits and Bytes

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [20]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

In [21]:
# qlora_model = AutoModelForSequenceClassification.from_pretrained(
#     foundation_model_name,
#     num_labels=2,
#     quantization_config=bnb_config,
#     # device_map="auto",   # automatically place modules on GPU
#     id2label={0: "not spam", 1: "spam"},
#     label2id={"not spam": 0, "spam": 1},
# )

In [22]:
# def TrainAndValidate2(model, model_output_path, model_path):
#     model.to(device)
    
#     # Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
#     trainer = Trainer(
#         model=model,
#         args=TrainingArguments(
#             output_dir=model_output_path,
#             # Set the learning rate
#             learning_rate=2e-5,
#             # Set the per device train batch size and eval batch size
#             per_device_train_batch_size=batch_size,
#             per_device_eval_batch_size=batch_size,
#             # Evaluate and save the model after each epoch
#             evaluation_strategy="epoch",
#             # eval_strategy="epoch",
#             save_strategy="epoch",
#             num_train_epochs=train_epochs,
#             weight_decay=0.01,
#             load_best_model_at_end=True,
#         ),
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["test"],
#         tokenizer=tokenizer,
#         data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
    
    
#     model.to(device)
#     trainer.evaluate()
    
#     model.save_pretrained(model_path)

## Apply LoRA(PEFT)

In [23]:
# qlora_model = get_peft_model(qlora_model, lora_config)

# qlora_model.print_trainable_parameters()

# qlora_trainer = TrainAndValidate2(qlora_model, qlora_model_output_path, qlora_model_path)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [24]:
def evaluate_model(model, dataset, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    preds, labels = [], []

    for i in range(0, len(dataset), batch_size):
        batch = dataset[i : i + batch_size]

        # Tokenize this batch of text dynamically with padding + truncation
        inputs = tokenizer(
            batch["sms"],
            truncation=True,
            padding=True,
            return_tensors="pt",
            max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        preds += torch.argmax(outputs.logits, dim=1).cpu().tolist()
        labels += batch["label"]

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return acc, f1


In [25]:
eval_dataset=tokenized_dataset["test"]

In [26]:
foundation_acc, foundation_f1 = evaluate_model(foundation_model, eval_dataset, tokenizer)
lora_acc, lora_f1 = evaluate_model(lora_model, eval_dataset, tokenizer)
# qlora_acc, qlora_f1 = evaluate_model(qlora_model, eval_dataset, tokenizer)

In [27]:
print(f"Foundation Model - Accuracy: {foundation_acc:.4f}, F1: {foundation_f1:.4f}")
print(f"Lora Model - Accuracy: {lora_acc:.4f}, F1: {lora_f1:.4f}")
# # print(f"Qlora Model - Accuracy: {qlora_acc:.4f}, F1: {qlora_f1:.4f}")

Foundation Model - Accuracy: 0.9883, F1: 0.9547
Lora Model - Accuracy: 0.9883, F1: 0.9547


## Zip output

In [29]:

source_dir = os.path.join(root_dir, training_artifacts_dir)
archive_name = os.path.join(root_dir, archive_zip)
# Create the zip archive
shutil.make_archive(archive_name, 'zip', source_dir)

print(f"'{archive_name}.zip' created successfully from '{source_dir}'")

'./tmp/archive.zip' created successfully from './tmp/training_artifacts'
